# Best practice for plotting batch-runs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
from cellpy.exceptions import NullData
from cellpy import log
import holoviews as hv
%matplotlib inline
hv.extension('bokeh')
log.setup_logging(default_level="INFO")

### Setting file names and loading data

In [ ]:
my_data = cellreader.CellpyData()
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_data.load(filename)
my_data.set_mass(0.1)

## Creating the ica data

In [ ]:
df_ica_charge, df_ica_discharge = ica.dqdv_frames(my_data, split=True, tidy=False, voltage_resolution=0.01)
tidy_ica_charge, tidy_ica_discharge = ica.dqdv_frames(my_data, split=True, tidy=True, normalize=True,
                                                      normalizing_factor=1.0, normalizing_roof=500.0,
                                                      voltage_resolution=0.01, post_smoothing=True)
comb_ica = ica.dqdv_frames(my_data, split=False, tidy=False)
tidy_comb_ica = ica.dqdv_frames(my_data, split=False, tidy=True)

In [ ]:
# quick view
tidy_comb_ica.plot(x="voltage", y="dq")

In [ ]:
# tidy_comb_ica.groupby("cycle").plot(x="voltage", y="dq");

In [ ]:
def plot_and_combine(ica_frame, selected_cycles=None, ax=None, back_end="matplotlib",
                     palette=None, xlabel="voltage", ylabel="dq", **kwargs):
    """Plot dqdv for selected cycles and combine them into a wide dataframe.
    
    This function assumes a tidy ica-frame as input.
    """
    selected_frames = []
    keys = []
    
    if back_end == "matplotlib":
        print("using matplotlib")
        if ax is None:
            fig, ax = plt.subplots()
    
    if back_end == "holoviews":
        curves = dict()
    
    if selected_cycles is not None:
        ica_frame = ica_frame.loc[ica_frame["cycle"].isin(selected_cycles)]
        
    for c, dd in ica_frame.groupby("cycle"):
        plot_label = f"cycle {c:03}"
        frame_label = f"cycle_{c:03}"
        
        if back_end == "matplotlib":
            ax.plot(dd["voltage"], dd["dq"], label=plot_label)
            
        elif back_end == "holoviews":
            curve = hv.Curve(dd[["voltage", "dq"]], vdims=["dq"])
            if palette is not None:
                curve = curve.opts(color=palette)
            curves[plot_label] = curve
        
        selected_frames.append(dd[["voltage", "dq"]])
        keys.append(frame_label)
        
    if back_end == "matplotlib":
        ax.legend()
        
    elif back_end == "holoviews":
        ax = hv.NdOverlay(curves).opts(
            xlabel=xlabel, 
            ylabel=ylabel,
            **kwargs,
        )
        
    new_df = pd.concat(selected_frames, axis=1, keys=keys)
    return ax, new_df

In [ ]:
selected_cycles = [1, 2, 3, 10, 15, 22]
p, new_com_ica = plot_and_combine(
    tidy_comb_ica, selected_cycles, back_end="holoviews", width=600,
    palette=hv.Palette('Magma'),
    xlim=(0.0, 1.0),
)

In [ ]:
p

In [ ]:
new_com_ica.head()

In [ ]:
selected_cycles = [1, 2, 3, 10, 15, 22]
ax, new_com_ica = plot_and_combine(tidy_comb_ica, selected_cycles, back_end="matplotlib")

## Saving the results

In [ ]:
# new_com_ica.to_csv("example_ica_out.csv", sep=";")

In [ ]:
# %%output fig='html' filename='ica_test_out'
# p